<a href="https://colab.research.google.com/github/duongquangvinh/Fundamental-Machine-Learning-model/blob/main/pytorch_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Modules

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse
from tqdm import tqdm
import numpy as np

## Data Preprocess

In [2]:
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [ ]:
trainset = torchvision.datasets.CIFAR10(root="./CIFAR10", train=True, transform=transform_train, download=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2, drop_last=True)

testset = torchvision.datasets.CIFAR10(root="./CIFAR10", train=False, transform=transform_test, download=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2, drop_last=True)

## Define Model

In [4]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        # We will use 3 maxpool, therefore input size will decrease by a factor of 2 in 3 times (32->16->8->4)
        self.fc1 = nn.Linear(4*4*64, 500)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 4*4*64)
        x = F.relu(self.fc1(x))
        
        # Applying dropout b/t layers which exchange highest parameters. This is a good practice
        x = self.dropout1(x)
        x = self.fc2(x)
        return x

In [5]:
net = LeNet()
net = net.to("cuda")

## Training part

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [8]:
best_acc = 0
for epoch in range(0,20):
    net.train()
    running_loss = 0.0
    for batch_idx, (imgs, labels) in tqdm(enumerate(trainloader)):
        img = imgs.to("cuda")
        label = labels.to("cuda")
        optimizer.zero_grad()
        output = net(img)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Training loss: {running_loss/len(trainloader)}")

    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (imgs, labels) in tqdm(enumerate(testloader)):
            img = imgs.to("cuda")
            label = labels.to("cuda")
            optimizer.zero_grad()
            output = net(img)
            loss = criterion(output, label)
            test_loss += loss.item()
            _, predict = torch.max(output, 1)
            total += label.size(0)
            correct += (predict == label).sum().item()
    print(f"Test loss: {test_loss/len(testloader)}")
    print(f"Accuracy: {100*correct/total}")

    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/cifar10_ckpt.pth')
        best_acc = acc

781it [00:21, 35.81it/s]

Training loss: 1.7315740799018577



156it [00:02, 58.86it/s]

Test loss: 1.5529224352958875
Accuracy: 43.669871794871796
Saving..



781it [00:21, 35.89it/s]

...

Test loss: 0.9312937523310001
Accuracy: 67.48798076923077
Saving..



781it [00:21, 35.61it/s]

Training loss: 1.0193072417786728



156it [00:02, 57.11it/s]

Test loss: 0.9351646399650818
Accuracy: 66.89703525641026
